# Corrélation

## Mesures journalières : 2018-2019

-----------------------------

- **Auteur** : Tarik Boumaza
- **Date** : 21 Mai 2021

-----------------------------

Imports

In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import scipy.stats

## 1. Data

### 1.1 Pollution

#### Indicateur

In [2]:
df_pollution = {'no': 0, 'no2': 0, 'o3': 0, 'pm2,5': 0, 'pm10': 0}

pollution_filepath = '../../data/pollution/mesures/'

def read_pollution_file(polluant):
    filename = pollution_filepath + 'indicateur/' + polluant + '_indicateur.csv'
    temp_df = pd.read_csv(filename, sep=";")
    nom_lignes = []
    data = []
    for i in range(len(temp_df)):
        nom_lignes.append(temp_df.iloc[i,0])
        data.append(temp_df['indicateur'][i])
    ar = numpy.array(data)
    temp_df = pd.DataFrame(ar, index = nom_lignes, columns = ['indicateur'])
    return temp_df

In [3]:
for cle, valeur in df_pollution.items():
    df_pollution[cle] = read_pollution_file(cle)

##### Statistiques

Pour chacun des polluants, calcul des indicateurs stastistiques suivants:
- moyenne ('moyenne')
- 1er quartile ('q1')
- médiane ('q2')
- 3ème quartile ('q3')
- dernier décile ('d10')

In [4]:
filename = '../../data/pollution/stats.csv'
df_stats = pd.read_csv(filename, sep=";")

nom_lignes = []
data = []
for i in range(len(df_stats)):
    temp = []
    j = 1
    nom_lignes.append(df_stats.iloc[i,0])
    while (j < len(df_stats.iloc[i])):
        temp.append(df_stats.iloc[i,j])
        j = j + 1
    data.append(temp)
ar = numpy.array(data)
df_stats = pd.DataFrame(ar, index = nom_lignes, columns = ['moyenne','q1','mediane','q3','d10'])

In [5]:
df_stats

,moyenne,q1,mediane,q3,d10
no,29.18,11.67,19.57,34.41,62.62
no2,32.59,22.51,30.74,40.48,50.45
o3,51.88,34.40,51.40,67.86,81.91
"pm2,5",19.68,10.65,16.10,24.44,36.38
pm10,25.10,15.70,21.65,30.48,42.37


In [6]:
def selection_date(date_debut, date_fin):
    for cle,valeur in df_pollution.items():
        df_pollution[cle] = valeur.loc[date_debut:date_fin]

In [7]:
selection_date('01/01/2018', '31/12/2019')

----------------------------------------

### 1.2. Mortalité

In [8]:
mortalite_filepath = '../../data/mortalite/journaliers_2018-2019/quotidienne/'

df_mortalite = {'2018': 0, '2019': 0}

def read_mortalite_file(annee):
    filename = mortalite_filepath + annee + '_quotidienne.csv'
    temp_df = pd.read_csv(filename, sep=";")
    nom_lignes = []
    data = []
    for i in range(len(temp_df)):
        nom_lignes.append(temp_df.iloc[i,0])
        data.append(temp_df['nb_deces'][i])
    ar = numpy.array(data)
    temp_df = pd.DataFrame(ar, index = nom_lignes, columns = ['nb_deces'])
    return temp_df

In [9]:
for cle, valeur in df_mortalite.items():
    df_mortalite[cle] = read_mortalite_file(cle)

### 3. Corrélation

In [10]:
data = {}
j = 0
for cle_m, valeur_m in df_mortalite.items():
    for i in range(len(valeur_m.index)):
        d = valeur_m.index[i]
        data[d] = {}
        data[d]['nb_deces'] = valeur_m.loc[valeur_m.index[i]].iloc[0]
        for cle_p, valeur_p in df_pollution.items():
            data[d][cle_p] = valeur_p['indicateur'][j]
        j = j + 1

In [11]:
mortalite = []
for c, v in df_mortalite.items():
    for i in range(len(v)):
        mortalite.append(v['nb_deces'][i])
    
'''
@brief pour un polluant donné, on ne sélectionne que les jours avec indicateur > seuil
'''
def select_seuil(polluant, seuil, decalage = 0):
    nb_deces = []
    indicateur_seuil = []
    for i in range(len(df_pollution[polluant]) - decalage):
        if (df_pollution[polluant].iloc[i]['indicateur'] > seuil):
            nb_deces.append(mortalite[i + decalage])
            indicateur_seuil.append(df_pollution[polluant].iloc[i]['indicateur'])
    return indicateur_seuil, nb_deces

'''
@brief calcul de la moyenne glissante sur n jours
'''
def moyenne_glissante(tab, n):
    if not(n < len(tab)):
        exit()
    temp = (tab[0] + tab[1])/2
    moy = [float(tab[0]), float(temp)]
    i = 2
    while(i < n):
        temp = ((temp*(i)) + tab[i]) / (i+1)
        moy.append(temp)
        i = i + 1
    while(i < len(tab)):
        temp = temp - tab[i - n]/n + tab[i]/n
        moy.append(temp)
        i = i + 1
    return moy


'''
@brief pour le polluant, on ne sélectionne que les jours avec indicateur > seuil après avoir calculé la moyenne glissante sur nb_jours
'''
def seuil_moy_glissante(polluant, seuil, nb_jours = 4):
    tab_pollution = []
    for c, v in data.items():
        tab_pollution.append(v[polluant])
    tab_pollution = moyenne_glissante(tab_pollution,nb_jours)
    tab_mortalite = moyenne_glissante(mortalite,nb_jours)
    
    indicateur_seuil = []
    nb_deces = []
    i = 0
    
    for i in range(len(tab_pollution)):
        if (tab_pollution[i] > seuil):
            indicateur_temp = 0.0
            mortalite_temp = 0.0
            j = 0
            while((i + j) < len(tab_pollution) and j < nb_jours):
                indicateur_temp = indicateur_temp + tab_pollution[i+j]
                mortalite_temp = mortalite_temp + tab_mortalite[i+j]
                j = j + 1
            indicateur_seuil.append(round(indicateur_temp/nb_jours,2))
            nb_deces.append(round(mortalite_temp/nb_jours,2))
    return indicateur_seuil, nb_deces


'''
@brief pour le polluant, on ne sélectionne que les jours avec indicateur > seuil et on cumule les décès sur les nb_jours suivants
'''
def select_seuil_cumul(polluant, seuil, nb_jours = 4):
    tab_pollution = []
    for c, v in data.items():
        tab_pollution.append(v[polluant])
    
    indicateur_seuil = []
    nb_deces = []
    i = 0
    
    for i in range(len(tab_pollution)):
        if (tab_pollution[i] > seuil):
            indicateur_temp = 0.0
            mortalite_temp = 0.0
            j = 0
            while((i + j) < len(tab_pollution) and (j < nb_jours)):
                indicateur_temp = indicateur_temp + tab_pollution[i+j]
                mortalite_temp = mortalite_temp + mortalite[i+j]
                j = j + 1
            indicateur_seuil.append(indicateur_temp/nb_jours)
            nb_deces.append(mortalite_temp/nb_jours)
    return indicateur_seuil, nb_deces


def seuil_moy_decalage(polluant, seuil, nb_jours = 4):
    tab_pollution = []
    indicateur_seuil = []
    nb_deces = []
    
    for c, v in data.items():
        tab_pollution.append(v[polluant])
    tab_pollution = moyenne_glissante(tab_pollution,nb_jours)
    tab_mortalite = moyenne_glissante(mortalite,nb_jours)
    
    for i in range(nb_jours):
        del tab_pollution[(len(tab_pollution)) - i - 1]
        
    for i in range(len(tab_pollution)):
        if (tab_pollution[i] > seuil):
            indicateur_seuil.append(round(tab_pollution[i],2))
            nb_deces.append(round(mortalite[i + nb_jours],2))
    return indicateur_seuil, nb_deces

In [12]:
def plot_pollution_mortalite(polluant, nb_jours = 7):
    plt.clf()
    x = moyenne_glissante(df_pollution[polluant]['indicateur'], nb_jours)
    y = mortalite        
    fig, ax = plt.subplots()  # Create a figure and an axes.
    ax.scatter(x, y, label='point')
    ax.set_xlabel('indicateur pollution')  # Add an x-label to the axes.
    ax.set_ylabel('nombre décès')  # Add a y-label to the axes.
    titre = 'indicateur de pollution en moyenne glissante \'' + polluant + ' vs mortalité\'' 
    ax.set_title(titre)  # Add a title to the axes.
    ax.legend()  # Add a legend.
    plt.show()
    
    
def plot_pollution_mortalite_seuil(polluant, seuil, decalage = 0):
    plt.clf()
    fig, ax = plt.subplots()  # Create a figure and an axes.
    x,y = select_seuil(polluant, seuil, decalage)
    ax.scatter(x, y, label='point')
    ax.set_xlabel('indicateur pollution')  # Add an x-label to the axes.
    ax.set_ylabel('nombre décès')  # Add a y-label to the axes.
    titre = 'indicateur de pollution (seuil) \'' + polluant + ' vs mortalité\'' 
    ax.set_title(titre)  # Add a title to the axes.
    ax.legend()  # Add a legend.
    plt.show()
    
    
def plot_seuil_moy_glissante(polluant, seuil, nb_jours = 5):
    plt.clf()
    fig, ax = plt.subplots()  # Create a figure and an axes.
    x, y = seuil_moy_glissante(polluant, seuil, nb_jours)
    #print(scipy.stats.spearmanr(x, y))
    ax.scatter(x, y, label='(pollution,nb_deces)')
    ax.set_xlabel('indicateur pollution')  # Add an x-label to the axes.
    ax.set_ylabel('nombre décès')  # Add a y-label to the axes.
    titre = 'seuil polluant dépassé en moy glissante \'' + polluant + ' vs mortalité' 
    ax.set_title(titre)  # Add a title to the axes.
    ax.legend()  # Add a legend.
    plt.show()
    
    
def plot_select_seuil_cumul(polluant, seuil, nb_jours = 4):
    plt.clf()
    fig, ax = plt.subplots()  # Create a figure and an axes.
    x,y = select_seuil_cumul(polluant, seuil, nb_jours)
    #print(scipy.stats.spearmanr(x, y))
    ax.scatter(x, y, label='point')
    ax.set_xlabel('indicateur pollution')  # Add an x-label to the axes.
    ax.set_ylabel('nombre décès')  # Add a y-label to the axes.
    titre = 'indicateur de pollution \'' + polluant + ' vs cumulée sur \'' + str(nb_jours) 
    ax.set_title(titre)  # Add a title to the axes.
    ax.legend()  # Add a legend.
    plt.show()
    


def plot_seuil_moy_decalage(polluant, seuil, nb_jours = 4):
    x,y = seuil_moy_decalage(polluant, seuil, nb_jours)
    r, p = scipy.stats.spearmanr(x, y)
    if ((p < 0.05) and (r > 0.5) and (len(x) > 15)):
        print(polluant)
        print(seuil)
        print(nb_jours)
        print(len(x))
        print(scipy.stats.spearmanr(x, y))
        plt.clf()
        fig, ax = plt.subplots()  # Create a figure and an axes.
        ax.scatter(x, y, label='point')
        ax.set_xlabel('indicateur pollution')  # Add an x-label to the axes.
        ax.set_ylabel('nombre décès')  # Add a y-label to the axes.
        titre = 'indicateur de pollution \'' + polluant + ' vs cumulée sur \'' + str(nb_jours) 
        ax.set_title(titre)  # Add a title to the axes.
        ax.legend()  # Add a legend.
        plt.show()

In [13]:
def test_correlation(polluant, seuil, nb_jours, donnees_min = 10, fct='select_seuil'):
    params = {'r':0, 'p':0, 'nb_donnees':0, 'x':0, 'y':0}
    if (fct=='select_seuil'):
        x,y = select_seuil(polluant, seuil, nb_jours)
    elif (fct=='seuil_moy_glissante'):
        x,y = seuil_moy_glissante(polluant, nb_jours)
    elif (fct=='select_seuil_cumul'):
        x,y = select_seuil_cumul(polluant, seuil, nb_jours)
    else:
        exit()
    if (len(x) > donnees_min):
        r,p = scipy.stats.spearmanr(x, y)
        if (params['r'] < r and p < 0.05):
            params['r'] = r
            params['p'] = p
            params['nb_donnees'] = len(x)
            params['x'] = x
            params['y'] = y
    return params

In [14]:
def print_dict(dict_correlation):
    for c, v in dict_correlation.items():
        s = str(c) + '\n'
        s = s + '{ r=' + str(v['r']) + ' ; p=' + str(v['p']) 
        s = s + '\nseuil=' + str(v['seuil']) + ' ; nb_jours=' + str(v['nb_jours']) + ' (' + str(v['nb_donnees']) + ' donnees, avec '  + str(v['fct']) + ') }'
        print(s, end='\n\n')

def test_correlations(polluant, j_min, j_max, donnees_min = 20):
    params = {'polluant':polluant, 'r':0, 'p':0, 'seuil':0, 'nb_jours':0, 'nb_donnees':0, 'fct':0, 'x':0, 'y':0}
    for i in range(round(df_stats['d10'].loc[polluant]*2)):
        for j in range(j_max-j_min):
            for fct in ['select_seuil', 'select_seuil_cumul', 'seuil_moy_glissante', 'seuil_moy_decalage']:
                if (fct=='select_seuil'):
                    x,y = select_seuil(polluant, i+1, j+j_min+1)
                elif (fct=='seuil_moy_glissante'):
                    x,y = seuil_moy_glissante(polluant, i+1, j+j_min++1)
                elif (fct=='select_seuil_cumul'):
                    x,y = select_seuil_cumul(polluant, i+1, j+j_min+1)
                elif (fct=='seuil_moy_decalage'):
                    x,y = seuil_moy_decalage(polluant, i+1, j+j_min+1)
                if (len(x) > donnees_min):
                    r,p = scipy.stats.spearmanr(x, y)
                    if (params['r'] < r and p < 0.05):
                        params['r'] = round(r,5)
                        params['p'] = round(p,5)
                        params['seuil'] = i + 1
                        params['nb_jours'] = j + j_min + 1
                        params['nb_donnees'] = len(x)
                        params['fct'] = fct
                        params['x'] = x
                        params['y'] = y
    return params

def fortes_correlations(j_min, j_max):
    correlation_params = {}
    for p, v in df_pollution.items():
        correlation_params[p] = test_correlations(p, j_min, j_max)
    return correlation_params

In [15]:
toutes_correlations = {}
for i in range(5):
    toutes_correlations[i] = fortes_correlations(i+1, i+2)
    print_dict(toutes_correlations[i])
    print()

no
{ r=0.33528 ; p=0.0
seuil=7 ; nb_jours=2 (719 donnees, avec seuil_moy_glissante) }

no2
{ r=0.57766 ; p=0.00054
seuil=58 ; nb_jours=2 (32 donnees, avec seuil_moy_glissante) }

o3
{ r=0.52023 ; p=0.01094
seuil=104 ; nb_jours=2 (23 donnees, avec seuil_moy_decalage) }

pm2,5
{ r=0.44338 ; p=0.03875
seuil=31 ; nb_jours=2 (22 donnees, avec seuil_moy_decalage) }

pm10
{ r=0.40209 ; p=0.00752
seuil=33 ; nb_jours=2 (43 donnees, avec seuil_moy_decalage) }


no
{ r=0.41557 ; p=0.0
seuil=1 ; nb_jours=3 (730 donnees, avec seuil_moy_glissante) }

no2
{ r=0.51112 ; p=0.01268
seuil=58 ; nb_jours=3 (23 donnees, avec seuil_moy_glissante) }

o3
{ r=0.48221 ; p=0.00519
seuil=100 ; nb_jours=3 (32 donnees, avec select_seuil) }

pm2,5
{ r=0.4347 ; p=0.00358
seuil=27 ; nb_jours=3 (43 donnees, avec select_seuil) }

pm10
{ r=0.46222 ; p=0.02638
seuil=41 ; nb_jours=3 (23 donnees, avec select_seuil) }


no
{ r=0.46519 ; p=0.0
seuil=6 ; nb_jours=4 (730 donnees, avec seuil_moy_glissante) }

no2
{ r=0.51866 ; p=

In [16]:
best = {}
for p, v in df_pollution.items():
    best[p] = {'polluant': p, 'r':0.0, 'p':0.0, 'seuil':0, 'nb_jours':0, 'nb_donnees':0, 'fct':0}
    
for i, v_i in toutes_correlations.items():
    for p, v_p in df_pollution.items():
        if (best[p]['r'] < toutes_correlations[i][p]['r']):
            best[p]['r'] = toutes_correlations[i][p]['r']
            best[p]['p'] = toutes_correlations[i][p]['p']
            best[p]['seuil'] = toutes_correlations[i][p]['seuil']
            best[p]['nb_jours'] = toutes_correlations[i][p]['nb_jours']
            best[p]['nb_donnees'] = toutes_correlations[i][p]['nb_donnees']
            best[p]['fct'] = toutes_correlations[i][p]['fct']

for p, v in best.items():
    print(v)

{'polluant': 'no', 'r': 0.51513, 'p': 0.0, 'seuil': 6, 'nb_jours': 6, 'nb_donnees': 730, 'fct': 'seuil_moy_glissante'}
{'polluant': 'no2', 'r': 0.57766, 'p': 0.00054, 'seuil': 58, 'nb_jours': 2, 'nb_donnees': 32, 'fct': 'seuil_moy_glissante'}
{'polluant': 'o3', 'r': 0.65843, 'p': 0.00117, 'seuil': 108, 'nb_jours': 4, 'nb_donnees': 21, 'fct': 'select_seuil'}
{'polluant': 'pm2,5', 'r': 0.44933, 'p': 0.0, 'seuil': 1, 'nb_jours': 6, 'nb_donnees': 730, 'fct': 'seuil_moy_glissante'}
{'polluant': 'pm10', 'r': 0.5436, 'p': 1e-05, 'seuil': 31, 'nb_jours': 4, 'nb_donnees': 61, 'fct': 'select_seuil'}
